In [1]:
import json

In [3]:
sourceFile = 'outputs/transcripts/sjV7NNwm1GU.json'
with open(sourceFile, 'r') as f:
    data = json.load(f)

text_content = ' '.join([item['text'] for item in data])

with open('output.txt', 'w') as f:
    f.write(text_content)

In [14]:
import httpx
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

OLLAMA_ENDPOINT = "http://localhost:11434/api/generate"

def fix_punctuation(text):
    prompt = f"""Please add missing punctuation marks to this text, ensuring that all sentences are properly punctuated. Do not add, remove, or modify any words. Preserve the exact original word order and content. Only correct punctuation errors such as missing periods, commas, or other necessary marks. Do not summarize or paraphrase the text.

{text}

Return only the corrected text, do not include a preamble."""

    response = httpx.post(
        OLLAMA_ENDPOINT,
        json={
            "model": "llama3.2:latest",
            #"model": "ifioravanti/mistral-grammar-checker",
            "prompt": prompt,
            "stream": False
        },
        headers={"Content-Type": "application/json"},
        timeout=500
    )

    if response.status_code != 200:
        logger.error(f"Error {response.status_code}: {response.text}")
        return None

    return response.json()["response"].strip()

# Read the transcript file
with open("output.txt", "r") as file:
    transcript = file.read()

# Fix punctuation
corrected_transcript = fix_punctuation(transcript)

# Save the corrected transcript
if corrected_transcript is not None:
    with open("corrected_transcript.txt", "w") as file:
        file.write(corrected_transcript)
else:
    logger.error("Failed to correct punctuation.")


INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
